# Extract

**Extração de dados do banco de dados sqlite Financial.db disponível junto a este notebook**

In [1]:
# Importa pacotes necessários para as análises
import sqlite3
import pandas as pd

**Conectando ao banco**

In [3]:
# Cria a conexão com o banco de dados
conexao = sqlite3.connect("Financial_Sample.db")

In [4]:
cursor = conexao.cursor()

**Extraindo os dados**

In [6]:
# Cria uma query
query = cursor.execute("SELECT * FROM Financial;")

In [7]:
# Guarda o resultado da query na variável dados
dados = query.fetchall()
dados

[('Carretera', 1618.5, 3.0, 20.0, 0.0, 'Canada'),
 ('Carretera', 1321.0, 3.0, 20.0, 0.0, 'Germany'),
 ('Carretera', 2178.0, 3.0, 15.0, 0.0, 'France'),
 ('Carretera', 888.0, 3.0, 15.0, 0.0, 'Germany'),
 ('Carretera', 2470.0, 3.0, 15.0, 0.0, 'Mexico'),
 ('Carretera', 1513.0, 3.0, 350.0, 0.0, 'Germany'),
 ('Montana', 921.0, 5.0, 15.0, 0.0, 'Germany'),
 ('Montana', 2518.0, 5.0, 12.0, 0.0, 'Canada'),
 ('Montana', 1899.0, 5.0, 20.0, 0.0, 'France'),
 ('Montana', 1545.0, 5.0, 12.0, 0.0, 'Germany'),
 ('Montana', 2470.0, 5.0, 15.0, 0.0, 'Mexico'),
 ('Montana', 2665.5, 5.0, 125.0, 0.0, 'Canada'),
 ('Montana', 958.0, 5.0, 300.0, 0.0, 'Mexico'),
 ('Montana', 2146.0, 5.0, 7.0, 0.0, 'Germany'),
 ('Montana', 345.0, 5.0, 125.0, 0.0, 'Canada'),
 ('Montana', 615.0, 5.0, 15.0, 0.0, 'United States of America'),
 ('Paseo', 292.0, 10.0, 20.0, 0.0, 'Canada'),
 ('Paseo', 974.0, 10.0, 15.0, 0.0, 'Mexico'),
 ('Paseo', 2518.0, 10.0, 12.0, 0.0, 'Canada'),
 ('Paseo', 1006.0, 10.0, 350.0, 0.0, 'Germany'),
 ('Paseo',

In [8]:
# Define os nomes das colunas do dataset
colunas = ["Produto","Unidades_Vendidas","Custo_Producao","Preco_Venda","Descontos","Pais"]

In [9]:
# Cria o dataframe para ETL
df = pd.DataFrame(dados, columns=colunas)
df.head()

,Produto,Unidades_Vendidas,Custo_Producao,Preco_Venda,Descontos,Pais
0,Carretera,1618.5,3.0,20.0,0.0,Canada
1,Carretera,1321.0,3.0,20.0,0.0,Germany
2,Carretera,2178.0,3.0,15.0,0.0,France
3,Carretera,888.0,3.0,15.0,0.0,Germany
4,Carretera,2470.0,3.0,15.0,0.0,Mexico


# Transform

## Realizando Engenharia de Atributos

Afim de enriquecer o dataset extraído do banco de dados, serão criadas as features:  
- Custo total producao  
- Total vendido bruto
- Total vendido líquido
- Lucro
- Proporção de unidades vendidas

In [18]:
df["Custo_Total_Producao"] = df["Unidades_Vendidas"] * df["Custo_Producao"]
df["Total_Vendido_Bruto"] = df["Unidades_Vendidas"] * df["Preco_Venda"]
df["Total_Vendido_Liquido"] = (df["Unidades_Vendidas"] * df["Preco_Venda"]) - df["Descontos"]
df["Lucro"] = df["Total_Vendido_Liquido"] - df["Custo_Total_Producao"]
df["Proporcao_Unidades_Vendidas"] = round((df["Unidades_Vendidas"] / df.Unidades_Vendidas.sum())*100, 2)

In [19]:
# Visualiza o resultado 
df.head()

,Produto,Unidades_Vendidas,Custo_Producao,Preco_Venda,Descontos,Pais,Custo_Total_Producao,Total_Vendido_Bruto,Total_Vendido_Liquido,Proporcao_Unidades_Vendidas,Lucro
0,Carretera,1618.5,3.0,20.0,0.0,Canada,4855.5,32370.0,32370.0,0.14,27514.5
1,Carretera,1321.0,3.0,20.0,0.0,Germany,3963.0,26420.0,26420.0,0.12,22457.0
2,Carretera,2178.0,3.0,15.0,0.0,France,6534.0,32670.0,32670.0,0.19,26136.0
3,Carretera,888.0,3.0,15.0,0.0,Germany,2664.0,13320.0,13320.0,0.08,10656.0
4,Carretera,2470.0,3.0,15.0,0.0,Mexico,7410.0,37050.0,37050.0,0.22,29640.0


In [17]:
# Print para verificar que os dados das variáveis são diferentes
print(df.Total_Vendido_Bruto.sum())
print(df.Total_Vendido_Liquido.sum())

127931598.5
118726350.25999999


# Load

Aqui o dataset será carregado novamente no banco de dados de origem.

In [20]:
# Carrega o dataset no banco de dados Financial
df.to_sql("financial_melhorado", conexao, index=False)

700

In [21]:
# Fecha a conexão com o banco
cursor.close()
conexao.close()